In [ ]:
import torchvision
import torch
import matplotlib.pyplot as plt
import cv2
import numpy as np
from IPython.display import display, Image, clear_output
import ipywidgets as widgets
import threading as thread
import time
import traitlets
import dataset
import os
from uuid import uuid1
import json
import glob
from datetime import datetime


In [ ]:
import cv2
import threading
import queue
import ipywidgets as widgets
from IPython.display import display, clear_output

# Create image widgets for displaying video feeds
image_widget = widgets.Image(format='jpeg', width=640, height=360)
image_widget2 = widgets.Image(format='jpeg', width=640, height=360)
on = False

# Initialize video capture for two cameras with lower resolution and FPS
cap = cv2.VideoCapture(0)
#cap.set(cv2.CAP_PROP_FPS, 15)

#cap2 = cv2.VideoCapture(1)
#cap2.set(cv2.CAP_PROP_FPS, 15)

# Create a toggle button to start/stop the feed
toggle_button = widgets.ToggleButton(value=False, description='Start/Stop Video')

# Frame queues for reducing latency
frame_queue = queue.Queue(maxsize=1)
frame_queue2 = queue.Queue(maxsize=1)

# Frame update function for the first camera
def update_image():
    global on
    while on:
        ret, frame = cap.read()
        if ret:
            # Add the latest frame to the queue, replacing the old one
            if not frame_queue.empty():
                frame_queue.get_nowait()
            frame_queue.put_nowait(frame)

# Frame update function for the second camera
def update_image2():
    global on
    while on:
        ret2, frame2 = cap2.read()
        if ret2:
            if not frame_queue2.empty():
                frame_queue2.get_nowait()
            frame_queue2.put_nowait(frame2)

# Display function to update widgets
def display_image():
    global on
    while on:
        if not frame_queue.empty():
            frame = frame_queue.get_nowait()
            success, jpeg = cv2.imencode('.jpeg', frame, [int(cv2.IMWRITE_JPEG_QUALITY), 80])
            if success:
                image_widget.value = jpeg.tobytes()
        if not frame_queue2.empty():
            frame2 = frame_queue2.get_nowait()
            success, jpeg2 = cv2.imencode('.jpeg', frame2, [int(cv2.IMWRITE_JPEG_QUALITY), 80])
            if success:
                image_widget2.value = jpeg2.tobytes()

# Start/stop the video feed
def toggle_video(change):
    global on
    if change['new']:
        on = True
        toggle_button.description = 'Stop Video'
        display(image_widget, image_widget2)
        
        # Start camera threads
        threading.Thread(target=update_image).start()
        threading.Thread(target=update_image2).start()
        
        # Start display thread
        threading.Thread(target=display_image).start()
    else:
        on = False
        toggle_button.description = 'Start Video'
        
        # Release video captures and clear outputs
        cap.release()
        cap2.release()
        clear_output()
        display(toggle_button)

# Observe button toggle to start/stop video feed
toggle_button.observe(toggle_video, 'value')

# Display the button
display(toggle_button)


In [ ]:
import os
import cv2
import numpy as np
import ipywidgets as widgets
from matplotlib import pyplot as plt
from matplotlib.widgets import RectangleSelector
from PIL import Image
from IPython.display import display

# Create a directory for storing data
def create_data_dirs(classes, root_dir='independentstudyjow/datasets/dataset1/data'):
    if not os.path.exists(root_dir):
        os.makedirs(root_dir)

    # Create subfolders for each class
    for class_name in classes:
        class_dir = os.path.join(root_dir, class_name)
        if not os.path.exists(class_dir):
            os.makedirs(class_dir)
            print(f"Created directory: {class_dir}")

# Example classes
classes = ['Philips', 'Hex', 'Star', 'Flat']
create_data_dirs(classes)

dropdown = widgets.Dropdown(
    options=classes,
    value=classes[0],
    description='Class:',
    disabled=False,
)


# Function to save the image to the directory with the class name
# def save_snapshot(directory):
#     image_path = os.path.join(directory, str(uuid1()) + '.jpg')
#     with open(image_path, 'wb') as f:
#         f.write(image_widget.value)

def save_snapshot(class_name):
    root_dir = 'independentstudyjow/datasets/dataset1/data/'
    directory = os.path.join(root_dir,class_name+"/"+str(uuid1())+'.jpg')
    # Save image from widget's byte data
    with open(directory, "wb") as f:
        f.write(image_widget.value)  # Save widget image data directly
    
    print(f'Saved: {directory}')

def save_snapshot2(class_name):
    root_dir = 'independentstudyjow/datasets/dataset1/data/'
    directory = os.path.join(root_dir,class_name+"/"+str(uuid1())+'.jpg')
    # Save image from widget's byte data
    with open(directory, "wb") as f:
        f.write(image_widget2.value)  # Save widget image data directly
    
    print(f'Saved: {directory}')

# Step 5: Capture images based on dropdown value (class label)
def on_dropdown_change(change):
    selected_class = change['new']
    print(f"Selected class: {selected_class}")

# Step 6: Connect the dropdown widget to the function
dropdown.observe(on_dropdown_change, names='value')

# Display the dropdown and image widgets
camera_widget = widgets.HBox([
image_widget,image_widget2
])
display(camera_widget)

# Save the dataset with buttons
def on_save_button_clicked(b):
    selected_class = dropdown.value
    if image_widget.value:  # If an image is loaded in the widget
        save_snapshot(selected_class)
    else:
        print("No image to save!")

def on_save_button_clicked2(b):
    selected_class = dropdown.value
    if image_widget2.value:  # If an image is loaded in the widget
        save_snapshot2(selected_class)
    else:
        print("No image to save!")

# Create save button
save_button = widgets.Button(description='Save Image')
save_button.on_click(on_save_button_clicked)
save_button2 = widgets.Button(description='Save Image 2nd Cam')
save_button2.on_click(on_save_button_clicked2)
# Add count widgets for specific class (Optional)
data_collection_widget = widgets.VBox([
    widgets.HBox([save_button, save_button2]),
    dropdown
])

display(data_collection_widget)


In [ ]:
import os
import torchvision
import torch
import matplotlib.pyplot as plt
import cv2
import numpy as np
from IPython.display import display, Image, clear_output
import ipywidgets as widgets
import threading as thread
import time
import traitlets
import dataset
from uuid import uuid1
import json
import glob
from datetime import datetime
HOME = os.getcwd()
print(HOME)

import ultralytics
ultralytics.checks()

from ultralytics import YOLO
from PIL import Image
import requests

model = YOLO('yolo11q.pt')
model2 = YOLO('yolo11s.pt')

%cd {HOME}
!mkdir datasets
%cd {HOME}/datasets

from roboflow import Roboflow

ROBOFLOW_API_KEY = ('14W8hOR0bWFloIDQecFa')
rf = Roboflow(api_key=ROBOFLOW_API_KEY)
project2 = rf.workspace("screw-sorting-project").project("side-screw-sorter")
version2 = project2.version(4)
dataset2 = version2.download("yolov11")

project = rf.workspace("screw-sorting-project").project("screw-sorting")
version = project.version(4)
dataset = version.download("yolov11")


In [ ]:
%cd {HOME}

!yolo task=detect mode=train model=yolo11s.pt data={dataset.location}/data.yaml epochs=10 imgsz=640 plots=True

In [ ]:
from IPython.display import Image as IPyImage

IPyImage(filename=f'{HOME}/runs/detect/train5/val_batch0_pred.jpg', width=600)

In [ ]:
!yolo task=detect mode=val model={HOME}/runs/detect/train5/weights/best.pt data={dataset.location}/data.yaml

In [ ]:
!yolo task=detect mode=predict model={HOME}/runs/detect/train5/weights/best.pt conf=0.25 source={dataset.location}/test/images save=True

In [ ]:
import cv2
import supervision as sv
import numpy as np
from threading import Thread
from queue import Queue
import torch
import inference
import ipywidgets as widgets
from IPython.display import display

# Select the device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Initialize Model
model_id = project.id.split("/")[1] + "/" + dataset.version
model_id2 = project2.id.split("/")[1] + "/" + dataset2.version
model = inference.get_model(model_id, '14W8hOR0bWFloIDQecFa')
model2 = inference.get_model(model_id2, '14W8hOR0bWFloIDQecFa')

# Initialize webcam
cap = cv2.VideoCapture(0)  # First webcam
cap2 = cv2.VideoCapture(2)  # Second webcam

# Image widgets for displaying frames
image_widget = widgets.Image(format='jpeg', width=640, height=360)
image_widget2 = widgets.Image(format='jpeg', width=640, height=360)
display(image_widget, image_widget2)

# Queue to store frames
frame_queue = Queue(maxsize=5)
frame_queue2 = Queue(maxsize=5)

# Flag to control the webcam loop
is_running = True

# Frame capture function for the first webcam
def capture_frames():
    global is_running
    while is_running:
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture image from webcam 1.")
            break
        if not frame_queue.full():
            frame_queue.put(frame)

# Frame capture function for the second webcam
def capture_frames1():
    global is_running
    while is_running:
        ret, frame2 = cap2.read()
        if not ret:
            print("Failed to capture image from webcam 2.")
            break
        if not frame_queue2.full():
            frame_queue2.put(frame2)

# Frame processing and inference function
xCoordinate = 0
yCoordinate = 0
labels = 0
xCoordinate2 = 0
yCoordinate2 = 0
labels2 = 0
def process_and_display():
    global is_running
    global xCoordinate
    global yCoordinate
    global labels
    global xCoordinate2
    global yCoordinate2
    global labels2
    while is_running:
        # Process first webcam frames
        if not frame_queue.empty:
            frame = frame_queue.get()
            results = model.infer(frame, confidence=0.4, overlap=0.3)[0]
            detections = sv.Detections.from_inference(results)
            # Annotate boxes and labels
            box_annotator = sv.BoxAnnotator()
            label_annotator = sv.LabelAnnotator()
            annotated_frame = box_annotator.annotate(scene=frame, detections=detections)
            annotated_frame = label_annotator.annotate(scene=annotated_frame, detections=detections)
            # Update the widget
            _, jpeg = cv2.imencode('.jpg', annotated_frame)
            image_widget.value = jpeg.tobytes()

        if not frame_queue2.empty():
            frame2 = frame_queue2.get()
            results = model2.infer(frame2, confidence=0.4, overlap=0.3)[0]
            detections2 = sv.Detections.from_inference(results)
            # Annotate boxes and labels
            box_annotator = sv.BoxAnnotator()
            label_annotator = sv.LabelAnnotator()

            annotated_frame2 = box_annotator.annotate(scene=frame2, detections=detections2)
            annotated_frame2 = label_annotator.annotate(scene=annotated_frame2, detections=detections2)
            _, jpeg2 = cv2.imencode('.jpg', annotated_frame2)
            image_widget2.value = jpeg2.tobytes()

            
            if(detections2.xyxy.size != 0&detections.xyxy.size != 0):
                print("Detections: ")
                for i in range(len(detections2.xyxy)):
                    labels2 = detections2.data.get('class_name', None)
                    xCoordinate2 = (detections2.xyxy[i][2]+detections2.xyxy[i][0])/2
                    yCoordinate2 = (detections2.xyxy[i][3]+detections2.xyxy[i][1])/2
                    for n in range(len(detections2.xyxy)):
                        if(n!=i):
                            if(abs(((detections2.xyxy[i][2]+detections2.xyxy[i][0])/2)-((detections2.xyxy[n][2]+detections2.xyxy[n][0])/2))<=5):
                                print("x: ", xCoordinate2, "y: ", yCoordinate2, labels2[i], labels2[n])
                        for s in range(len(detections.xyxy)):
                            if(s!=i):
                                if(abs(((detections.xyxy[i][2]+detections.xyxy[i][0])/2)-((detections2.xyxy[s][2]+detections2.xyxy[s][0])/2))<=5):
                                    print("x: ", xCoordinate, "y: ", yCoordinate, labels[s],labels2[i], labels2[n])
                        n=len(detections2.xyxy)

                        
        print("match not found")
    

    # Release cameras when done
    cap.release()
    cap2.release()
    print("Webcam feed stopped.")

# Start the capture and processing threads
capture_thread = Thread(target=capture_frames)
capture_thread1 = Thread(target=capture_frames1)
process_thread = Thread(target=process_and_display)

capture_thread.start()
capture_thread1.start()
process_thread.start()

# To stop the loops, set is_running = False in your notebook


In [ ]:
pip install numpy

In [ ]:
!nvidia-smi

In [ ]:
import torch
print("CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)


In [ ]:
import torch

print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)

# Test GPU usage
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using device:", device)
    tensor = torch.rand(3, 3).to(device)
    print("Tensor allocated on:", tensor.device)
else:
    print("CUDA is not available. Check your installation.")